In [ ]:
## Amir Pourmand
## This is done! 

In [ ]:
import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F

import numpy as np

In [ ]:
batch_size = 128
num_epochs = 2
device = torch.device('cpu')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return x

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output.log(), target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output.log(), target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

## Shap Implementation

In [ ]:
import pickle
import inspect
import logging
import numpy as np
import cloudpickle

log = logging.getLogger('shap')

class Serializable():

    def save(self, out_file):
        pickle.dump(type(self), out_file)

    @classmethod
    def load(cls, in_file, instantiate=True):
        if instantiate:
            return cls._instantiated_load(in_file)
        return {}

    @classmethod
    def _instantiated_load(cls, in_file, **kwargs):
        obj_type = pickle.load(in_file)
        if obj_type is None:
            return None

        if not inspect.isclass(obj_type) or (not issubclass(obj_type, cls) and (obj_type is not cls)):
            raise Exception(f"Invalid object type loaded from file. {obj_type} is not a subclass of {cls}.")

        # here we call the constructor with all the arguments we have loaded
        constructor_args = obj_type.load(in_file, instantiate=False, **kwargs)
        used_args = inspect.getfullargspec(obj_type.__init__)[0]
        return obj_type(**{k: constructor_args[k] for k in constructor_args if k in used_args})


class Serializer():
    def __init__(self, out_stream, block_name, version):
        self.out_stream = out_stream
        self.block_name = block_name
        self.block_version = version
        self.serializer_version = 0 # update this when the serializer changes

    def __enter__(self):
        log.debug("serializer_version = %d", self.serializer_version)
        pickle.dump(self.serializer_version, self.out_stream)
        log.debug("block_name = %s", self.block_name)
        pickle.dump(self.block_name, self.out_stream)
        log.debug("block_version = %d", self.block_version)
        pickle.dump(self.block_version, self.out_stream)
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        log.debug("END_BLOCK___")
        pickle.dump("END_BLOCK___", self.out_stream)

    def save(self, name, value, encoder="auto"):
        log.debug("name = %s", name)
        pickle.dump(name, self.out_stream)
        if encoder is None or encoder is False:
            log.debug("encoder_name = %s", "no_encoder")
            pickle.dump("no_encoder", self.out_stream)
        elif callable(encoder):
            log.debug("encoder_name = %s", "custom_encoder")
            pickle.dump("custom_encoder", self.out_stream)
            encoder(value, self.out_stream)
        elif encoder == ".save" or (isinstance(value, Serializable) and encoder == "auto"):
            log.debug("encoder_name = %s", "serializable.save")
            pickle.dump("serializable.save", self.out_stream)
            if len(inspect.getfullargspec(value.save)[0]) == 3: # backward compat for MLflow, can remove 4/1/2021
                value.save(self.out_stream, value)
            else:
                value.save(self.out_stream)
        elif encoder == "auto":
            if isinstance(value, (int, float, str)):
                log.debug("encoder_name = %s", "pickle.dump")
                pickle.dump("pickle.dump", self.out_stream)
                pickle.dump(value, self.out_stream)
            else:
                log.debug("encoder_name = %s", "cloudpickle.dump")
                pickle.dump("cloudpickle.dump", self.out_stream)
                cloudpickle.dump(value, self.out_stream)
        else:
            raise ValueError(f"Unknown encoder type '{encoder}' given for serialization!")
        log.debug("value = %s", str(value))

class Deserializer():
    """ Load data items from an input stream.
    """

    def __init__(self, in_stream, block_name, min_version, max_version):
        self.in_stream = in_stream
        self.block_name = block_name
        self.block_min_version = min_version
        self.block_max_version = max_version

        # update these when the serializer changes
        self.serializer_min_version = 0
        self.serializer_max_version = 0

    def __enter__(self):

        # confirm the serializer version
        serializer_version = pickle.load(self.in_stream)
        log.debug("serializer_version = %d", serializer_version)
        if serializer_version < self.serializer_min_version:
            raise ValueError(
                f"The file being loaded was saved with a serializer version of {serializer_version}, " + \
                f"but the current deserializer in SHAP requires at least version {self.serializer_min_version}."
            )
        if serializer_version > self.serializer_max_version:
            raise ValueError(
                f"The file being loaded was saved with a serializer version of {serializer_version}, " + \
                f"but the current deserializer in SHAP only support up to version {self.serializer_max_version}."
            )

        # confirm the block name
        block_name = pickle.load(self.in_stream)
        log.debug("block_name = %s", block_name)
        if block_name != self.block_name:
            raise ValueError(
                f"The next data block in the file being loaded was supposed to be {self.block_name}, " + \
                f"but the next block found was {block_name}."
            )

        # confirm the block version
        block_version = pickle.load(self.in_stream)
        log.debug("block_version = %d", block_version)
        if block_version < self.block_min_version:
            raise ValueError(
                f"The file being loaded was saved with a block version of {block_version}, " + \
                f"but the current deserializer in SHAP requires at least version {self.block_min_version}."
            )
        if block_version > self.block_max_version:
            raise ValueError(
                f"The file being loaded was saved with a block version of {block_version}, " + \
                f"but the current deserializer in SHAP only support up to version {self.block_max_version}."
            )
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        # confirm the block end token
        for _ in range(100):
            end_token = pickle.load(self.in_stream)
            log.debug("end_token = %s", end_token)
            if end_token == "END_BLOCK___":
                return
            else:
                self._load_data_value()
        raise ValueError(
            f"The data block end token wsa not found for the block {self.block_name}."
        )

    def load(self, name, decoder=None):
        """ Load a data item from the current input stream.
        """
        # confirm the block name
        loaded_name = pickle.load(self.in_stream)
        log.debug("loaded_name = %s", loaded_name)
        print("loaded_name", loaded_name)
        if loaded_name != name:
            raise ValueError(
                f"The next data item in the file being loaded was supposed to be {name}, " + \
                f"but the next block found was {loaded_name}."
            ) # We should eventually add support for skipping over unused data items in old formats...

        value = self._load_data_value(decoder)
        log.debug("value = %s", str(value))
        return value

    def _load_data_value(self, decoder=None):
        encoder_name = pickle.load(self.in_stream)
        log.debug("encoder_name = %s", encoder_name)
        if encoder_name == "custom_encoder" or callable(decoder):
            assert callable(decoder), "You must provide a callable custom decoder for the data item {name}!"
            return decoder(self.in_stream)
        if encoder_name == "no_encoder":
            return None
        if encoder_name == "serializable.save":
            return Serializable.load(self.in_stream)
        if encoder_name == "numpy.save":
            return np.load(self.in_stream)
        if encoder_name == "pickle.dump":
            return pickle.load(self.in_stream)
        if encoder_name == "cloudpickle.dump":
            return cloudpickle.load(self.in_stream)

        raise ValueError(f"Unsupported encoder type found: {encoder_name}")
{"mode":"full","isActive":False}

class Masker(Serializable):

    def __call__(self, mask, *args):
        '''super class of all maskers'''
       

In [ ]:

import numba

@numba.jit
def identity(x):
    return x
@numba.jit
def _identity_inverse(x):
    return x
identity.inverse = _identity_inverse

@numba.jit
def logit(x):
    return np.log(x/(1-x))
@numba.jit
def _logit_inverse(x):
    return 1/(1+np.exp(-x))
logit.inverse = _logit_inverse

def safe_isinstance(obj, class_path_str):
    if isinstance(class_path_str, str):
        class_path_strs = [class_path_str]
    elif isinstance(class_path_str, list) or isinstance(class_path_str, tuple):
        class_path_strs = class_path_str
    else:
        class_path_strs = ['']
    
    # try each module path in order
    for class_path_str in class_path_strs:
        if "." not in class_path_str:
            raise ValueError("class_path_str must be a string or list of strings specifying a full \
                module path to a class. Eg, 'sklearn.ensemble.RandomForestRegressor'")

        module_name, class_name = class_path_str.rsplit(".", 1)

        if module_name not in sys.modules:
            continue

        module = sys.modules[module_name]
        
        #Get class
        _class = getattr(module, class_name, None)
        
        if _class is None:
            continue
        
        if isinstance(obj, _class):
            return True

    return False

import time
import tqdm


class ShowProgress():
    def __init__(self, iterable, total, desc, silent, start_delay):
        self.iter = iter(iterable)
        self.start_time = time.time()
        self.pbar = None
        self.total = total
        self.desc = desc
        self.start_delay = start_delay
        self.silent = silent
        self.unshown_count = 0
    
    def __next__(self):
        if self.pbar is None and time.time() - self.start_time > self.start_delay:
           self.pbar = tqdm.tqdm(total=self.total, initial=self.unshown_count, desc=self.desc, disable=self.silent)
           self.pbar.start_t = self.start_time
        if self.pbar is not None:
            self.pbar.update(1)
        else:
            self.unshown_count += 1
        try:
            return next(self.iter)
        except StopIteration as e:
            if self.pbar is not None:
                self.pbar.close()
            raise e

    def __iter__(self):
        return self

def show_progress(iterable, total=None, desc=None, silent=False, start_delay=10):
    return ShowProgress(iterable, total, desc, silent, start_delay)


In [ ]:

class Model(Serializable):

    def __init__(self, model=None):
        """ Wrap a callable model as a SHAP Model object.
        """
        if isinstance(model, Model):
            self.inner_model = model.inner_model
        else:
            self.inner_model = model

    def __call__(self, *args):
        return self.inner_model(*args)

    def save(self, out_file):
        """ Save the model to the given file stream.
        """
        super().save(out_file)
        with Serializer(out_file, "shap.Model", version=0) as s:
            s.save("model", self.inner_model)

    @classmethod
    def load(cls, in_file, instantiate=True):
        if instantiate:
            return cls._instantiated_load(in_file)

        kwargs = super().load(in_file, instantiate=False)
        with Deserializer(in_file, "shap.Model", min_version=0, max_version=0) as s:
            kwargs["model"] = s.load("model")
        return kwargs

In [ ]:

class TeacherForcing(Model):

    def __init__(self, model, tokenizer=None, similarity_model=None, similarity_tokenizer=None, batch_size=128, device=None):
        super().__init__(model)

        self.tokenizer = tokenizer
        # set pad token if not defined
        if self.tokenizer is not None and self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.device = device
        self.batch_size = batch_size
        # assign text generation function
        if safe_isinstance(model, "transformers.PreTrainedModel") or safe_isinstance(model, "transformers.TFPreTrainedModel"):
            self.text_generate = models.TextGeneration(self.inner_model, tokenizer=self.tokenizer, device=self.device)
            self.similarity_model = model
            self.similarity_tokenizer = tokenizer
            self.model_agnostic = False
        else:
            self.text_generate = models.TextGeneration(self.inner_model, device=self.device)
            self.similarity_model = similarity_model
            self.similarity_tokenizer = similarity_tokenizer
            # set pad token for a similarity tokenizer(in a model agnostic scenario) if not defined
            if self.similarity_tokenizer is not None and self.similarity_tokenizer.pad_token is None:
                self.similarity_tokenizer.pad_token = self.similarity_tokenizer.eos_token
            self.model_agnostic = True
        # initializing target which is the target sentence/ids for every new row of explanation
        self.output = None
        self.output_names = None

        self.similarity_model_type = None
        if safe_isinstance(self.similarity_model, "transformers.PreTrainedModel"):
            self.similarity_model_type = "pt"
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if self.device is None else self.device
            self.similarity_model = self.similarity_model.to(self.device)
        elif safe_isinstance(self.similarity_model, "transformers.TFPreTrainedModel"):
            self.similarity_model_type = "tf"

    def __call__(self, X, Y):
        output_batch = None
        # caching updates output names and target sentence ids
        self.update_output_names(Y[:1])
        start_batch_idx, end_batch_idx = 0, len(X)
        while start_batch_idx < end_batch_idx:
            X_batch = X[start_batch_idx:start_batch_idx+self.batch_size]
            Y_batch = Y[start_batch_idx:start_batch_idx+self.batch_size]
            logits = self.get_teacher_forced_logits(X_batch, Y_batch)
            logodds = self.get_logodds(logits)
            if output_batch is None:
                output_batch = logodds
            else:
                output_batch = np.concatenate((output_batch, logodds))
            start_batch_idx += self.batch_size
        return output_batch

    def update_output_names(self, output):
        # check if the target sentence has been updated (occurs when explaining a new row)
        if (self.output is None) or (not np.array_equal(self.output, output)):
            self.output = output
            self.output_names = self.get_output_names(output)

    def get_output_names(self, output):
        output_ids = self.get_outputs(output)
        output_names = [self.similarity_tokenizer.decode([x]).strip() for x in output_ids[0, :]]
        return output_names

    def get_outputs(self, X):
        # check if output is a sentence or already parsed target ids
        if X.dtype.type is np.str_:
            parsed_tokenizer_dict = parse_prefix_suffix_for_tokenizer(self.similarity_tokenizer)
            keep_prefix, keep_suffix = parsed_tokenizer_dict['keep_prefix'], parsed_tokenizer_dict['keep_suffix']
            if keep_suffix > 0:
                output_ids = np.array(self.similarity_tokenizer(X.tolist(), padding=True)["input_ids"])[:, keep_prefix:-keep_suffix]
            else:
                output_ids = np.array(self.similarity_tokenizer(X.tolist(), padding=True)["input_ids"])[:, keep_prefix:]
        else:
            output_ids = X
        return output_ids

    def get_inputs(self, X, padding_side='right'):
        if self.model_agnostic:
            input_sentences = np.array(self.inner_model(X))
        else:
            input_sentences = np.array(X)
        self.similarity_tokenizer.padding_side = padding_side
        inputs = self.similarity_tokenizer(input_sentences.tolist(), return_tensors=self.similarity_model_type, padding=True)
        self.similarity_tokenizer.padding_side = 'right'
        return inputs

    def get_logodds(self, logits):
        if self.output.dtype.type is np.str_:
            output_ids = self.get_outputs(self.output)[0]
        else:
            output_ids = self.output[0]

        def calc_logodds(arr):
            probs = np.exp(arr) / np.exp(arr).sum(-1)
            logodds = sp.special.logit(probs)
            return logodds

        logodds = np.apply_along_axis(calc_logodds, -1, logits)
        logodds_for_output_ids = logodds[:, np.array(range(logodds.shape[1])), output_ids]
        return logodds_for_output_ids

    def model_inference(self, inputs, output_ids):
        if self.similarity_model_type == "pt":
            inputs = inputs.to(self.device)
            output_ids = torch.tensor(output_ids, dtype=torch.int64, device=self.device)
            self.similarity_model.eval()
            with torch.no_grad():
                if self.similarity_model.config.is_encoder_decoder:
                    outputs = self.similarity_model(**inputs, decoder_input_ids=output_ids, labels=output_ids, return_dict=True)
                else:
                    inputs["input_ids"] = torch.cat((inputs["input_ids"], output_ids), dim=-1)
                    attention_mask_for_output_ids = torch.ones(output_ids.shape, dtype=output_ids.dtype, device=self.device)
                    inputs["attention_mask"] = torch.cat((inputs["attention_mask"], attention_mask_for_output_ids), dim=-1)
                    inputs["position_ids"] = (inputs["attention_mask"].long().cumsum(-1) - 1)
                    inputs["position_ids"].masked_fill_(inputs["attention_mask"] == 0, 0)
                    outputs = self.similarity_model(**inputs, return_dict=True)
                logits = outputs.logits.detach().cpu().numpy().astype('float64')
        elif self.similarity_model_type == "tf":
            output_ids = tf.convert_to_tensor(output_ids, dtype=tf.int32)
            if self.similarity_model.config.is_encoder_decoder:
                if self.device is None:
                    outputs = self.similarity_model(inputs, decoder_input_ids=output_ids, labels=output_ids, return_dict=True)
                else:
                    try:
                        with tf.device(self.device):
                            outputs = self.similarity_model(inputs, decoder_input_ids=output_ids, labels=output_ids, return_dict=True)
                    except RuntimeError as e:
                        print(e)
            else:
                inputs["input_ids"] = tf.concat((inputs["input_ids"], output_ids), axis=-1)
                attention_mask_for_output_ids = tf.ones(output_ids.shape, dtype=output_ids.dtype)
                inputs["attention_mask"] = tf.concat((inputs["attention_mask"], attention_mask_for_output_ids), axis=-1)
                inputs["position_ids"] = tf.math.cumsum(inputs["attention_mask"], axis=-1) - 1
                inputs["position_ids"] = tf.where(inputs["attention_mask"] == 0, 0, inputs["position_ids"])
                if self.device is None:
                    outputs = self.similarity_model(inputs, return_dict=True)
                else:
                    try:
                        with tf.device(self.device):
                            outputs = self.similarity_model(inputs, return_dict=True)
                    except RuntimeError as e:
                        print(e)
            logits = outputs.logits.numpy().astype('float64')
        return logits

    def get_teacher_forced_logits(self, X, Y):
        if (hasattr(self.similarity_model.config, "is_encoder_decoder") and not self.similarity_model.config.is_encoder_decoder) \
            and (hasattr(self.similarity_model.config, "is_decoder") and not self.similarity_model.config.is_decoder):
            raise ValueError(
                "Please assign either of is_encoder_decoder or is_decoder to True in model config for extracting target sentence ids"
            )
        output_ids = self.get_outputs(Y)
        if self.similarity_model.config.is_encoder_decoder:
            inputs = self.get_inputs(X, padding_side='right')
            decoder_start_token_id = None
            if hasattr(self.similarity_model.config, "decoder_start_token_id") and \
                    self.similarity_model.config.decoder_start_token_id is not None:
                decoder_start_token_id = self.similarity_model.config.decoder_start_token_id
            elif hasattr(self.similarity_model.config, "bos_token_id") and self.similarity_model.config.bos_token_id is not None:
                decoder_start_token_id = self.similarity_model.config.bos_token_id
            elif (hasattr(self.similarity_model.config, "decoder") and hasattr(self.similarity_model.config.decoder, "bos_token_id") and \
                    self.similarity_model.config.decoder.bos_token_id is not None):
                decoder_start_token_id = self.similarity_model.config.decoder.bos_token_id
            else:
                raise ValueError(
                    "No decoder_start_token_id or bos_token_id defined in config for encoder-decoder generation"
                )
            output_start_id = np.ones((output_ids.shape[0], 1)) * decoder_start_token_id
            output_ids = np.concatenate((output_start_id, output_ids), axis=-1)
            logits = self.model_inference(inputs, output_ids)
            logits = logits[:, :-1, :]
        else:
            inputs = self.get_inputs(X, padding_side='left')
            logits = self.model_inference(inputs, output_ids)
            logits = logits[:, -output_ids.shape[1]-1:-1, :]
        return logits

    def save(self, out_file):
        super().save(out_file)

        with Serializer(out_file, "shap.models.TeacherForcing", version=0) as s:
            s.save("tokenizer", self.tokenizer)
            s.save("similarity_model", self.similarity_model)
            s.save("similarity_tokenizer", self.similarity_tokenizer)
            s.save("batch_size", self.batch_size)
            s.save("device", self.device)

    @classmethod
    def load(cls, in_file, instantiate=True):
        if instantiate:
            return cls._instantiated_load(in_file)

        kwargs = super().load(in_file, instantiate=False)
        with Deserializer(in_file, "shap.models.TeacherForcing", min_version=0, max_version=0) as s:
            kwargs["tokenizer"] = s.load("tokenizer")
            kwargs["similarity_model"] = s.load("similarity_model")
            kwargs["similarity_tokenizer"] = s.load("similarity_tokenizer")
            kwargs["batch_size"] = s.load("batch_size")
            kwargs["device"] = s.load("device")
        return kwargs

In [ ]:
import copy
import numpy as np
import scipy as sp

class Explainer(Serializable):

    def __init__(self, model, masker=None, link=None, algorithm="auto", output_names=None, feature_names=None, **kwargs):
        self.model = model
        self.output_names = output_names
        self.feature_names = feature_names

        # wrap the incoming masker object as a shap.Masker object
        if safe_isinstance(masker, "pandas.core.frame.DataFrame") or \
                ((safe_isinstance(masker, "numpy.ndarray") or sp.sparse.issparse(masker)) and len(masker.shape) == 2):
            if algorithm == "partition":
                self.masker = maskers.Partition(masker)
            else:
                self.masker = maskers.Independent(masker)
        elif safe_isinstance(masker, ["transformers.PreTrainedTokenizer", "transformers.tokenization_utils_base.PreTrainedTokenizerBase"]):
            if (safe_isinstance(self.model, "transformers.PreTrainedModel") or safe_isinstance(self.model, "transformers.TFPreTrainedModel")) and \
                    safe_isinstance(self.model, MODELS_FOR_SEQ_TO_SEQ_CAUSAL_LM + MODELS_FOR_CAUSAL_LM):
                self.masker = maskers.Text(masker, mask_token="...", collapse_mask_token=True)
            else:
                self.masker = maskers.Text(masker)
        elif (masker is list or masker is tuple) and masker[0] is not str:
            self.masker = maskers.Composite(*masker)
        elif (masker is dict) and ("mean" in masker):
            self.masker = maskers.Independent(masker)
        elif masker is None and isinstance(self.model, models.TransformersPipeline):
            return self.__init__( # pylint: disable=non-parent-init-called
                self.model, self.model.inner_model.tokenizer,
                link=link, algorithm=algorithm, output_names=output_names, feature_names=feature_names, **kwargs
            )
        else:
            self.masker = masker

        if safe_isinstance(self.model, "transformers.pipelines.Pipeline"):
            return self.__init__( 
                models.TransformersPipeline(self.model), self.masker,
                link=link, algorithm=algorithm, output_names=output_names, feature_names=feature_names, **kwargs
            )

        if (safe_isinstance(self.model, "transformers.PreTrainedModel") or safe_isinstance(self.model, "transformers.TFPreTrainedModel")) and \
                safe_isinstance(self.model, MODELS_FOR_SEQ_TO_SEQ_CAUSAL_LM + MODELS_FOR_CAUSAL_LM):
            self.model = models.TeacherForcing(self.model, self.masker.tokenizer)
            self.masker = maskers.OutputComposite(self.masker, self.model.text_generate)
        elif safe_isinstance(self.model, "shap.models.TeacherForcing") and safe_isinstance(self.masker, ["shap.maskers.Text", "shap.maskers.Image"]):
            self.masker = maskers.OutputComposite(self.masker, self.model.text_generate)
        elif safe_isinstance(self.model, "shap.models.TopKLM") and safe_isinstance(self.masker, "shap.maskers.Text"):
            self.masker = maskers.FixedComposite(self.masker)

        if callable(link) and callable(getattr(link, "inverse", None)):
            self.link = link
        else:
            raise Exception("The passed link function needs to be callable and have a callable .inverse property!")

        if self.__class__ is Explainer:

            if algorithm == "auto":

                if explainers.Linear.supports_model_with_masker(model, self.masker):
                    algorithm = "linear"
                elif explainers.Tree.supports_model_with_masker(model, self.masker): 
                    algorithm = "tree"
                elif explainers.Additive.supports_model_with_masker(model, self.masker):
                    algorithm = "additive"

                elif callable(self.model):
                    if issubclass(type(self.masker), maskers.Independent):
                        if self.masker.shape[1] <= 10:
                            algorithm = "exact"
                        else:
                            algorithm = "permutation"
                    elif issubclass(type(self.masker), maskers.Partition):
                        if self.masker.shape[1] <= 32:
                            algorithm = "exact"
                        else:
                            algorithm = "permutation"
                    elif issubclass(type(self.masker), maskers.Composite):
                        if getattr(self.masker, "partition_tree", None) is None:
                            algorithm = "permutation"
                        else:
                            algorithm = "partition" # TODO: should really only do this if there is more than just tabular
                    elif issubclass(type(self.masker), maskers.Image) or issubclass(type(self.masker), maskers.Text) or \
                            issubclass(type(self.masker), maskers.OutputComposite) or issubclass(type(self.masker), maskers.FixedComposite):
                        algorithm = "partition"
                    else:
                        algorithm = "permutation"

                # if we get here then we don't know how to handle what was given to us
                else:
                    raise Exception("The passed model is not callable and cannot be analyzed directly with the given masker! Model: " + str(model))

            # build the right subclass
            if algorithm == "exact":
                self.__class__ = explainers.Exact
                explainers.Exact.__init__(self, self.model, self.masker, link=self.link, feature_names=self.feature_names, **kwargs)
            elif algorithm == "permutation":
                self.__class__ = explainers.Permutation
                explainers.Permutation.__init__(self, self.model, self.masker, link=self.link, feature_names=self.feature_names, **kwargs)
            elif algorithm == "partition":
                self.__class__ = explainers.Partition
                explainers.Partition.__init__(self, self.model, self.masker, link=self.link, feature_names=self.feature_names, output_names=self.output_names, **kwargs)
            elif algorithm == "tree":
                self.__class__ = explainers.Tree
                explainers.Tree.__init__(self, self.model, self.masker, link=self.link, feature_names=self.feature_names, **kwargs)
            elif algorithm == "additive":
                self.__class__ = explainers.Additive
                explainers.Additive.__init__(self, self.model, self.masker, link=self.link, feature_names=self.feature_names, **kwargs)
            elif algorithm == "linear":
                self.__class__ = explainers.Linear
                explainers.Linear.__init__(self, self.model, self.masker, link=self.link, feature_names=self.feature_names, **kwargs)
            else:
                raise Exception("Unknown algorithm type passed: %s!" % algorithm)


    def __call__(self, *args, max_evals="auto", main_effects=False, error_bounds=False, batch_size="auto",
                 outputs=None, silent=False, **kwargs):
      
        if issubclass(type(self.masker), maskers.OutputComposite) and len(args)==2:
            self.masker.model = models.TextGeneration(target_sentences=args[1])
            args = args[:1]
        num_rows = None
        args = list(args)
        if self.feature_names is None:
            feature_names = [None for _ in range(len(args))]
        elif issubclass(type(self.feature_names[0]), (list, tuple)):
            feature_names = copy.deepcopy(self.feature_names)
        else:
            feature_names = [copy.deepcopy(self.feature_names)]
        for i in range(len(args)):

           
            if num_rows is None:
                try:
                    num_rows = len(args[i])
                except Exception: 
                    pass

            if safe_isinstance(args[i], "pandas.core.frame.DataFrame"):
                feature_names[i] = list(args[i].columns)
                args[i] = args[i].to_numpy()

            if safe_isinstance(args[i], "nlp.arrow_dataset.Dataset"):
                args[i] = args[i]["text"]
            elif issubclass(type(args[i]), dict) and "text" in args[i]:
                args[i] = args[i]["text"]

        if batch_size == "auto":
            if hasattr(self.masker, "default_batch_size"):
                batch_size = self.masker.default_batch_size
            else:
                batch_size = 10

        values = []
        output_indices = []
        expected_values = []
        mask_shapes = []
        main_effects = []
        hierarchical_values = []
        clustering = []
        output_names = []
        if callable(getattr(self.masker, "feature_names", None)):
            feature_names = [[] for _ in range(len(args))]
        for row_args in show_progress(zip(*args), num_rows, self.__class__.__name__+" explainer", silent):
            row_result = self.explain_row(
                *row_args, max_evals=max_evals, main_effects=main_effects, error_bounds=error_bounds,
                batch_size=batch_size, outputs=outputs, silent=silent, **kwargs
            )
            values.append(row_result.get("values", None))
            output_indices.append(row_result.get("output_indices", None))
            expected_values.append(row_result.get("expected_values", None))
            mask_shapes.append(row_result["mask_shapes"])
            main_effects.append(row_result.get("main_effects", None))
            clustering.append(row_result.get("clustering", None))
            hierarchical_values.append(row_result.get("hierarchical_values", None))
            output_names.append(row_result.get("output_names", None))

            if callable(getattr(self.masker, "feature_names", None)):
                row_feature_names = self.masker.feature_names(*row_args)
                for i in range(len(row_args)):
                    feature_names[i].append(row_feature_names[i])

        arg_values = [[] for a in args]
        for i, v in enumerate(values):
            pos = 0
            for j in range(len(args)):
                mask_length = np.prod(mask_shapes[i][j])
                arg_values[j].append(values[i][pos:pos+mask_length])
                pos += mask_length

        expected_values = pack_values(expected_values)
        main_effects = pack_values(main_effects)
        output_indices = pack_values(output_indices)
        main_effects = pack_values(main_effects)
        hierarchical_values = pack_values(hierarchical_values)
        clustering = pack_values(clustering)

        ragged_outputs = False
        if output_indices is not None:
            ragged_outputs = not all(len(x) == len(output_indices[0]) for x in output_indices)
        if self.output_names is None:
            if None not in output_names:
                if not ragged_outputs:
                    sliced_labels = np.array(output_names)
                else:
                    sliced_labels = [np.array(output_names[i])[index_list] for i,index_list in enumerate(output_indices)]
            else:
                sliced_labels = None
        else:
            labels = np.array(self.output_names)
            sliced_labels = [labels[index_list] for index_list in output_indices]
            if not ragged_outputs:
                sliced_labels = np.array(sliced_labels)

        if isinstance(sliced_labels, np.ndarray) and len(sliced_labels.shape) == 2:
            if np.all(sliced_labels[0,:] == sliced_labels):
                sliced_labels = sliced_labels[0]

        out = []
        for j in range(len(args)):

            tmp = []
            for i, v in enumerate(arg_values[j]):
                if np.prod(mask_shapes[i][j]) != np.prod(v.shape):
                    tmp.append(v.reshape(*mask_shapes[i][j], -1))
                else:
                    tmp.append(v.reshape(*mask_shapes[i][j]))
            arg_values[j] = pack_values(tmp)

            if hasattr(self.masker, "data_transform"):
                data = pack_values([self.masker.data_transform(v) for v in args[j]])
            else:
                data = args[j]

            out.append(Explanation(
                arg_values[j], expected_values, data,
                feature_names=feature_names[j], main_effects=main_effects,
                clustering=clustering,
                hierarchical_values=hierarchical_values,
                output_names=sliced_labels 
            ))
        return out[0] if len(out) == 1 else out

    def explain_row(self, *row_args, max_evals, main_effects, error_bounds, outputs, silent, **kwargs):
      
        
        return {}

    @staticmethod
    def supports_model_with_masker(model, masker):
        
        return False

    @staticmethod
    def _compute_main_effects(fm, expected_value, inds):
       

        # mask each input on in isolation
        masks = np.zeros(2*len(inds)-1, dtype=np.int)
        last_ind = -1
        for i in range(len(inds)):
            if i > 0:
                masks[2*i - 1] = -last_ind - 1 # turn off the last input
            masks[2*i] = inds[i] # turn on this input
            last_ind = inds[i]

        # compute the main effects for the given indexes
        main_effects = fm(masks) - expected_value

        # expand the vector to the full input size
        expanded_main_effects = np.zeros(len(fm))
        for i, ind in enumerate(inds):
            expanded_main_effects[ind] = main_effects[i]

        return expanded_main_effects

    def save(self, out_file, model_saver=".save", masker_saver=".save"):
        super().save(out_file)
        with Serializer(out_file, "shap.Explainer", version=0) as s:
            s.save("model", self.model, model_saver)
            s.save("masker", self.masker, masker_saver)
            s.save("link", self.link)

    @classmethod
    def load(cls, in_file, model_loader=Model.load, masker_loader=Masker.load, instantiate=True):
      
        if instantiate:
            return cls._instantiated_load(in_file, model_loader=model_loader, masker_loader=masker_loader)

        kwargs = super().load(in_file, instantiate=False)
        with Deserializer(in_file, "shap.Explainer", min_version=0, max_version=0) as s:
            kwargs["model"] = s.load("model", model_loader)
            kwargs["masker"] = s.load("masker", masker_loader)
            kwargs["link"] = s.load("link")
        return kwargs

def pack_values(values):

    # collapse the values if we didn't compute them
    if values is None or values[0] is None:
        return None
    elif np.issubdtype(type(values[0]), np.number) or len(np.unique([len(v) for v in values])) == 1:
        return np.array(values)
    else:
        return np.array(values, dtype=np.object)

In [ ]:
class Deep(Explainer):
    def __init__(self, model, data, session=None, learning_phase_flags=None):
        
        # first, we need to find the framework
        if type(model) is tuple:
            a, b = model
            try:
                a.named_parameters()
                framework = 'pytorch'
            except:
                framework = 'tensorflow'
        else:
            try:
                model.named_parameters()
                framework = 'pytorch'
            except:
                framework = 'tensorflow'

        if framework == 'tensorflow':
            self.explainer = TFDeep(model, data, session, learning_phase_flags)
        elif framework == 'pytorch':
            self.explainer = PyTorchDeep(model, data)

        self.expected_value = self.explainer.expected_value

    def shap_values(self, X, ranked_outputs=None, output_rank_order='max', check_additivity=True):
        
        return self.explainer.shap_values(X, ranked_outputs, output_rank_order, check_additivity=check_additivity)

## Shap Usage

In [ ]:
# since shuffle=True, this is a random sample of test data
batch = next(iter(test_loader))
images, _ = batch

background = images[:100]
test_images = images[100:103]

e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(test_images)

In [ ]:
shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
test_numpy = np.swapaxes(np.swapaxes(test_images.numpy(), 1, -1), 1, 2)

In [ ]:
shap.image_plot(shap_numpy, -test_numpy)